*Import the google drive to save and load data*

In [1]:
import os
print("Files in the current directory:", os.listdir('/content'))
from google.colab import drive
drive.mount('/content/drive')

Files in the current directory: ['.config', 'sample_data']
Mounted at /content/drive


*After that upload trhe Testing and Training folder to the given path*

In [4]:
import numpy as np
X_train = np.load('/content/drive/MyDrive/MySentinel/Dataset/Training/training_features.npy')
y_train = np.load('/content/drive/MyDrive/MySentinel/Dataset/Training/training_labels.npy')
X_test = np.load('/content/drive/MyDrive/MySentinel/Dataset/Testing/testing_features.npy')
y_test = np.load('/content/drive/MyDrive/MySentinel/Dataset/Testing/testing_labels.npy')

*Let's see how many data we are working with, usually we have 4times more for training than testing*

In [10]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.dtype, y_train.dtype, X_test.dtype, y_test.dtype)

(12000, 5, 5, 4)
(12000,)
(3000, 5, 5, 4)
(3000,)
uint16 int32 float32 int32


*Let's take a closer look*

In [12]:
print(X_train[0])


[0 0 0 ... 4 4 4]
